<a href="https://colab.research.google.com/github/christianhidber/easyagents/blob/master/jupyter_notebooks/workshop/easyagents_line_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Line World

* an agent lives in a finite linear world of uneven elements
* at each moment it is in a certain position
* initial position is the middle
* some positions gain rewards, some don't
* rewards are between 0 and 15
* agent can either move left or right
* Objective: maximize total reward = sum(rewards) + sum(steps)
* Cost per step: -1
* Done Condition: agent is at pos 0 or total reward <= -20

<img src='https://github.com/christianhidber/easyagents/blob/master/jupyter_notebooks/workshop/line-world.png?raw=1'>

### Install gym, tensorflow, tf-agents,..., setup display

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
if 'google.colab' in sys.modules:
    !pip install easyagents >/dev/null
    !pip install -q --ignore-installed tensorflow-probability==0.7.0 2>/dev/null
    !pip install -q networkx==2.3.0 

**=> Restart notebook to activate the downgraded tensorflow-probability dependency.**

In [2]:
import easyagents
easyagents.__version__

##### suppress package warnings, in colab: load additional packages for rendering

In [3]:
import sys
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
if 'google.colab' in sys.modules:
    !apt-get install xvfb >/dev/null
    !pip install pyvirtualdisplay >/dev/null    
    
    from pyvirtualdisplay import Display
    Display(visible=0, size=(960, 720)).start() 

# Define Gym Environment

## What you need to define
1. Actions
1. Observation
1. Reward
1. Game done?

## Remember how an environment looks like

```python
class MyEnv(gym.Env):
  def __init__(self):
      # set up the environment

    def step(self, action):
        # make the action have an impact on the environment
        # and return the information the algorithms need
        return observation, reward, done, info

    def reset(self):
        # reset this environment to the initial state and return the according observation
        return observation

    def render(self, mode='ansi'):
        # spits out a human readable rendering of the environment
        if mode == 'ansi':
            return self._render_ansi()
        if mode == 'human':
            return self._render_human()            
        elif mode == 'rgb_array':
            return self._render_rgb() 
```

https://gym.openai.com/

https://github.com/openai/gym/blob/master/docs/creating-environments.md


## Good sanity check

_Would you as a human be able to play the game based on the observation and reward you get?_


In [0]:
import numpy as np
import math

import gym
from gym import error, spaces, utils

import matplotlib.pyplot as plt
from IPython.display import display, clear_output

LEFT = 0
RIGHT = 1

class LineWorldEnv(gym.Env):

    # constructor sets up the properties of the environment
    # most important action space, observation space and reward range
    def __init__(self, world=[10, 0, 0, 5, 0, 2, 15]):
        self.world = np.array(world)
        # the agent can perform  different actions
        number_of_actions = 2
        self.action_space = spaces.Discrete(number_of_actions)

        self.size_of_world = len(world)
        
        # STEP II
        # this needs to match size and range of what you return in
        # get_observation 
        # definition of world is a list in self.world
        # the length of the world is in self.size_of_world

        # CHANGE HERE START

        self.max_reward = 15
        self.min_reward = 0
        # CHANGE TWO: machting size of observation  
        self.observation_size = self.observation_size = 1 + self.size_of_world

        # CHANGE HERE END
        
        # the environment's current state is described by the position of the agent and the remaining rewards
        low = np.full(self.observation_size, self.min_reward)
        high = np.full(self.observation_size, self.max_reward)
        self.observation_space = spaces.Box(low=low,
                                            high=high,
                                            dtype=np.float32)

        self.reward_range = (-1, 1)
        # 32 is only theoretical, because we need to travel at least 9 steps, leaving us with 23 practically
        self.optimum = self.world.sum() - 9

        self._figure = None

        self.reset()

    # STEP I
    # position: [self.pos]
    # remaining rewards: self.remaining_rewards
    # if you want to append two lists/arrays use np.append(list1, list2)

    # CHANGE HERE START

    def get_observation(self):
        # CHANGE ONE: observation passed to agent is now meaningful
        # contains current position and all remaining rewards
        return np.append([self.pos], self.remaining_rewards)

    # CHANGE HERE END

    def reset(self):
        self.total_reward = 0
        self.done = False
        self.pos = math.floor(len(self.world) / 2)
        self.steps = 0

        self.remaining_rewards = np.array(self.world, copy=True)
        return self.get_observation()

    def step(self, action):
        if action == LEFT and self.pos != 0:
          self.pos -= 1
        elif self.pos < self.size_of_world -1:
          self.pos += 1

        reward = self.remaining_rewards[self.pos] - 1
        normalized_reward = reward / self.optimum
        self.total_reward += normalized_reward
        self.remaining_rewards[self.pos] = 0

        if self.pos == 0 or self.total_reward * self.optimum <= -20:
          self.done = True
        self.steps += 1

        observation = self.get_observation()
        info = None
        return observation, normalized_reward, self.done, info

    def _render_to_ansi(self):
        return 'position: {position}, remaining rewards: {rewards}, total reward so far: {total}, normalized total reward: {normalized_total}, steps so far: {steps}, game done: {done}'.format(
            position=self.pos, 
            rewards=self.remaining_rewards, 
            total=self.total_reward * self.optimum, 
            normalized_total = self.total_reward,
            done=self.done,
            steps=self.steps)              

    def _render_to_figure(self):
        """ Renders the current state as a graph with matplotlib """
        if self._figure is not None:
            plt.close(self._figure)
        self._figure, ax = plt.subplots(1, figsize=(8, 4))
        ax.set_ylim(bottom=-1, top=self.max_reward + 1)
        x = np.arange(0, self.size_of_world, 1, dtype=np.uint8)
        y = self.remaining_rewards
        plt.plot([self.pos, self.pos], [0, 2], 'r^-')
        ax.scatter(x, y, s=75)
        self._figure.canvas.draw()
        
    def _render_to_human(self):
        """ show render_to_figure in a jupyter cell. 
            the result of each call is rendered in the same cell"""
        clear_output(wait=True)
        self._render_to_figure()
        plt.pause(0.01)
        
    def _render_to_rgb(self):
        """ convert the output of render_to_figure to a rgb_array """
        self._render_to_figure()
        self._figure.canvas.draw()
        buf = self._figure.canvas.tostring_rgb()
        num_cols, num_rows = self._figure.canvas.get_width_height()
        plt.close(self._figure)
        self._figure = None
        result = np.fromstring(buf, dtype=np.uint8).reshape(num_rows, num_cols, 3)
        return result        

    def render(self, mode='ansi'):
        if mode == 'ansi':
            return self._render_to_ansi()
        elif mode == 'human':
            return self._render_to_human()
        elif mode == 'rgb_array':
            return self._render_to_rgb()
        else:
            super().render(mode=mode)


# Hands-On 1: Try out our environment

_use yourself to simulate the agent_

* Create an environment and play the game to the end manually using the cells below
* Use a sequence of left and right commands to find the best solution possible
* Show yourself only local observations and reward for each step and choose an action
* Would this be sufficent for a good reward?
* What reward do you get? What is the optimum?

In [0]:
env = LineWorldEnv()

In [6]:
# shows the raw observation, does look meaningful now 
env.reset()

In [7]:
# more complete and readable information
env.render()

In [8]:
env.step(LEFT)
env.render()

In [9]:
env.step(RIGHT)
env.render()

In [10]:
env.render(mode='human')

# Train policy with tfagents PpoAgent

In [11]:
%%time

from easyagents.env import register_with_gym
from easyagents.agents import PpoAgent
from easyagents.callbacks import log, plot

env_name = "LineWorld-v0"
register_with_gym(gym_env_name=env_name, entry_point=LineWorldEnv, max_episode_steps=100)

ppoAgent = PpoAgent(gym_env_name = env_name, fc_layers=(500,500,500))

plots = [plot.State(), plot.Actions(), plot.StepRewards(), plot.Rewards(), plot.Steps(), plot.ToMovie()]
ppoAgent.train(plots, 
               num_iterations = 25, 
               num_episodes_per_iteration = 10,
               max_steps_per_episode = 20)

# Hands-On 2: Define the observation in a way that allows the agent to train

* You see the agent does not learn with the observation given. Why is that?

* Find an observation that works and impement it in `get_observation`

*  You need to update the `observation_space` to the same shape of your observation at the same time.

* Use env.reset() to check the observation

* Make sure to re-register your environment before restarting the training

* Why does it work? Is this the best observation? How would you even know?

* Do you think this would be sufficient for a real world example? If not, what is missing?

 # STOP HERE

## Play (turn ouput into a movie)

In [12]:
ppoAgent.play([plot.State(), plot.StepRewards(num_steps_between_plot=1), plot.Actions(), plot.ToMovie(fps=3)], 
              num_episodes=5, max_steps_per_episode=20)

Play uses the previously trained policy. We play for 5 episodes, each with max 20 steps. All steps are rendered into a movie with a frame rate of 3 steps per second.

# How does Reinforcement Learning really work?

Presentation on slides

# Hands-On 3: Make your notebook work with Reinforce

* What parts do you need to change?

* How do the results compare to what you have seen with PPO?

* Can you tweak the parameters (like the size of the neural network) to improve the results?


In [13]:
from easyagents.agents import ReinforceAgent

reinforceAgent = ReinforceAgent(gym_env_name = env_name, fc_layers=(500,500,500))

plots = [plot.State(), plot.Actions(), plot.StepRewards(), plot.Rewards(), plot.Steps(), plot.ToMovie()]
reinforceAgent.train(plots, 
               num_iterations = 50, 
               num_episodes_per_iteration = 10,
               max_steps_per_episode = 20)

# Next Steps:

## Generalizing our Environment

* solving a single set up of our line world is not very realistic
* there should be variations to rewards, size, and starting position
* while training with with each episode we could randomly set rewards and starting postition
* different sizes could be simulated by using a fixed max size with a random number of padding entires (e.g -1) at the end



## Applying Reinforcement Learning to a problem

* Is your problem approachable by Reinforcement Learning (more on that later)?
  * Can you defined what would be the agent and what the environment?
* You can simulate your environment or are able to safely perform a large number of experiments in the real world
* You need to be able to define your problem as a Markov Decision Process (MDP)
* Good sanity check: Would you as a human be able to play the game based on the observation and reward you get?
* there are different kinds of observations
  * fully observed environment (e.g. Jump'n'Run Game)
  * partially observed environment (e.g. Ego Shooter)
* Choose a proper Reinforcement Learning Algorithm (more on that later)

## MDP: Markov Decision Process in Reinforcement Learning

* https://en.wikipedia.org/wiki/Markov_decision_process
* Model your observation in a way that which actions the agent took to get to a certain state are irrelevant
* That means all information necessary to derive a proper action need to be in a single observation
* The agents must not be forced to also learn the sequence of observations
* E.g. the position of the PacMan is not enough. Which path taken results in where the ghosts are and what pills have been eaten